In [ ]:
#merge data from multiple files into one file
import pandas as pd

#import data
umsatzdaten_gekuerzt = pd.read_csv("umsatzdaten_gekuerzt.csv")
kiwo = pd.read_csv("kiwo.csv")
wetter = pd.read_csv("wetter.csv")

#inspect data
print(umsatzdaten_gekuerzt.head())
print(umsatzdaten_gekuerzt.shape)
print(kiwo.head())
print(kiwo.shape)
print(wetter.head())
print(wetter.shape)

#merge wetter and umsatzdaten data
wetter_umsatzdaten = pd.merge(wetter, umsatzdaten_gekuerzt, on="Datum")
print(wetter_umsatzdaten.head())
print(wetter_umsatzdaten.shape)

#merge with kiwo data
wetter_umsatzdaten_kiwo = pd.merge(wetter_umsatzdaten, kiwo, on="Datum")
print(wetter_umsatzdaten_kiwo.head())
print(wetter_umsatzdaten_kiwo.shape)

#remove kiwo dates from data
wetter_umsatzdaten_except_kiwo = wetter_umsatzdaten[~wetter_umsatzdaten["Datum"].isin(kiwo["Datum"])]
print(wetter_umsatzdaten_except_kiwo.head())
print(wetter_umsatzdaten_except_kiwo.shape)

#save data
wetter_umsatzdaten.to_csv("wetter_umsatzdaten.csv", index=False)
wetter_umsatzdaten_kiwo.to_csv("wetter_umsatzdaten_kiwo.csv", index=False)  
wetter_umsatzdaten_except_kiwo.to_csv("wetter_umsatzdaten_except_kiwo.csv", index=False)    

#inspect Umsatzdaten_gekuerzt data for missing dates
print("Missing values per column:")
print(wetter_umsatzdaten.isnull().sum())

#visualize data
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Convert the "Datum" column to datetime
wetter_umsatzdaten['Datum'] = pd.to_datetime(wetter_umsatzdaten['Datum'], format='%Y-%m-%d')
wetter_umsatzdaten['Wochentag'] = wetter_umsatzdaten['Datum'].dt.day_name()

# Order weekdays
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create bar chart with confidence intervals
plt.figure(figsize=(12, 6))
sns.barplot(x='Wochentag', 
            y='Umsatz', 
            data=wetter_umsatzdaten, 
            order=weekday_order,
            ci=95,  # Add 95% confidence intervals
            capsize=0.1)  # Add caps to error bars

plt.title('Mean Sales per Weekday with 95% Confidence Intervals')
plt.ylabel('Mean Sales')
plt.xlabel('Weekday')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.show()


#a scatterplot of a
